In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import time
import zipfile

In [ ]:
# Autenticación con Google Drive
def authenticate_drive():
    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(r"D:\Users\u_sociales\Desktop\Extracción MINAGRI\client_secret_479070046140-73lf85vftkp4mp69edh0pa0rchp54ga1.apps.googleusercontent.com.json")
    gauth.LocalWebserverAuth()
    return GoogleDrive(gauth)

drive = authenticate_drive()

# Buscar o crear carpeta en Drive
def get_or_create_folder(drive, folder_name, parent_id):
    query = f"title = '{folder_name}' and '{parent_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
    folder_list = drive.ListFile({'q': query}).GetList()
    if folder_list:
        return folder_list[0]['id']
    folder = drive.CreateFile({
        'title': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [{'id': parent_id}]
    })
    folder.Upload()
    print(f"📁 Carpeta creada en Drive: {folder_name}")
    return folder['id']

# Extraer archivos .tif desde .zip
def extract_tif_files(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    tif_extraidos = []

    for archivo in os.listdir(input_folder):
        if archivo.lower().endswith(".zip"):
            ruta_zip = os.path.join(input_folder, archivo)
            base_name = os.path.splitext(archivo)[0]
            try:
                with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
                    for nombre in zip_ref.namelist():
                        if nombre.lower().endswith(".tif"):
                            nuevo_nombre = f"{base_name}_{os.path.basename(nombre)}"
                            ruta_salida = os.path.join(output_folder, nuevo_nombre)
                            with zip_ref.open(nombre) as origen, open(ruta_salida, "wb") as destino:
                                destino.write(origen.read())
                            tif_extraidos.append(ruta_salida)
            except Exception as e:
                print(f"❌ Error al extraer {archivo}: {e}")
    return tif_extraidos

# Subir archivos .tif a Google Drive
def upload_tif_files(drive, folder_path, parent_folder_id):
    for archivo in os.listdir(folder_path):
        if archivo.lower().endswith(".tif"):
            ruta = os.path.join(folder_path, archivo)
            file_drive = drive.CreateFile({'title': archivo, 'parents': [{'id': parent_folder_id}]})
            file_drive.SetContentFile(ruta)
            file_drive.Upload()
            print(f"✅ Subido: {archivo}")
            time.sleep(1)

# Ruta local base
carpeta_base = r"C:\Users\Public\Downloads"

# ID de carpeta raíz en Drive
ROOT_FOLDER_ID = "1plTbfdcORgnF1ECWN7RgMD5tUeBHoYw5"

# Mapeo entre carpetas locales (con prefijo S_) y carpetas abreviadas en Drive
mapeo_drive = {
    "S_2015_12_2016_05": "2016_05",
    "S_2016_06_2016_11": "2016_11",
    "S_2016_12_2017_05": "2017_05",
    "S_2017_06_2017_11": "2017_11",
    "S_2017_12_2018_05": "2018_05",
    "S_2018_06_2018_11": "2018_11",
    "S_2018_12_2019_05": "2019_05",
    "S_2019_06_2019_11": "2019_11",
    "S_2019_12_2020_05": "2020_05",  
    "S_2020_06_2020_08": "2020_08"}

# 8. Procesar cada carpeta
for carpeta_local, carpeta_drive in mapeo_drive.items():
    ruta_zip = os.path.join(carpeta_base, carpeta_local)
    ruta_extraido = os.path.join(carpeta_base, f"{carpeta_local}_EXTRAIDO")

    print(f"\n📦 Extrayendo ZIPs desde: {carpeta_local}")
    tifs = extract_tif_files(ruta_zip, ruta_extraido)

    if tifs:
        print(f"🟢 {len(tifs)} archivos .tif extraídos.")
        folder_id = get_or_create_folder(drive, carpeta_drive, ROOT_FOLDER_ID)
        upload_tif_files(drive, ruta_extraido, folder_id)
        print(f"☁️ Archivos subidos a carpeta: {carpeta_drive}")
    else:
        print(f"⚠️ No se encontraron archivos .tif en: {carpeta_local}")